### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import aomap
from survey_tools import healpix

### Options

In [ ]:
config = aomap.read_config('config.yaml')

ao_system_name = 'GNAO-Optimal'
dec_limit = [-20,60]
map_level = 6

ao_system = next((system for system in config.ao_systems if system['name'] == ao_system_name), None)

In [ ]:
print(f"Map Resolution: {healpix.get_resolution(map_level).to(u.deg):.2f}")
print(f"Map Area: {healpix.get_area(map_level).to(u.deg**2):.2f}")

### Asterism Coverage

In [ ]:
map_data = aomap.get_map_data(config, map_level, f"asterism-coverage-{ao_system_name}", survey='ews', dec_limit=dec_limit)
mean_coverage = np.mean(map_data.values)
print(f'Average Coverage over EWS within Dec Limit: {mean_coverage:.1%}')

min_coverage = 0.50
total_pix = len(map_data.values)
pixs_filter = map_data.values >= min_coverage
num_pix = np.sum(pixs_filter)
print(f'Fraction of Pix with Coverage >= {min_coverage:.0%}: {num_pix/total_pix:.1%} ({num_pix}/{total_pix})')


In [ ]:
aomap.plot_map(aomap.get_map_data(config, map_level, f"asterism-coverage-{ao_system_name}"),
               milkyway=True, milkyway_width=20,
               surveys=[['ews', {'edgecolor': 'red', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-north', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-south', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-fornax', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}]],
               rotation=110, projection='cart', width=15, vmax=1.0
)

In [ ]:
aomap.plot_map(aomap.get_map_data(config, map_level, f"asterism-coverage-{ao_system_name}", survey='ews', dec_limit=dec_limit),
               milkyway=True, milkyway_width=20,
               surveys=[['ews', {'edgecolor': 'red', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-north', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-south', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-fornax', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}]],
               lines=[{'dec': dec_limit[0], 'ls': '--'}, {'dec': dec_limit[1], 'ls': '--'}],
               rotation=110, projection='cart', zoom=False, width=15, vmax=1.0
)

### Asterism Counts and Density

In [ ]:
map_data = aomap.get_map_data(config, map_level, f"asterism-count-{ao_system_name}")
total_asterisms = np.sum(map_data.values)
print(f'\nTotal Asterisms: {total_asterisms:,.0f}')

map_data = aomap.get_map_data(config, config.max_data_level, f"asterism-count-{ao_system_name}", survey='ews', dec_limit=dec_limit)
total_asterisms = np.sum(map_data.values)
print(f'Usable Asterisms: {total_asterisms:,.0f}')

In [ ]:
aomap.plot_map(aomap.get_map_data(config, map_level, f"asterism-count-{ao_system_name}"),
               milkyway=True, milkyway_width=20,
               surveys=[['ews', {'edgecolor': 'red', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-north', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-south', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-fornax', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}]],
               rotation=110, projection='cart', width=15, norm='symlog', vmax=1000
)

In [ ]:
aomap.plot_map(aomap.get_map_data(config, map_level, f"asterism-count-{ao_system_name}", survey='ews', dec_limit=dec_limit),
               milkyway=True, milkyway_width=20,
               surveys=[['ews', {'edgecolor': 'red', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-north', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-south', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}],
                        ['edf-fornax', {'edgecolor': 'pink', 'linewidth': 1.5, 'linestyle': 'solid'}]],
               lines=[{'dec': dec_limit[0], 'ls': '--'}, {'dec': dec_limit[1], 'ls': '--'}],
               rotation=110, projection='cart', zoom=False, width=15, norm='symlog', vmin=1, vmax=1000
)

### Plot Asterims

In [ ]:
level = 6
plot_type = 'dust' # dust or stars

pix = None
ra = None
dec = None
object_name = 'cosmos'

if pix is None:
    if ra is None or dec is None:
        match object_name:
            case 'cosmos':
                ra = 150.0
                dec = 2.0
            case 'omegacentauri':
                ra = 201.05
                dec = -47.48
            case 'galcenter':
                ra = 266.42
                dec = -29.00
            case 'andromeda':
                ra = 10.68
                dec = 41.27
            case 'lmc':
                ra = 80.89
                dec = -69.76 + 8
            case 'smc':
                ra = 13.19
                dec = -72.83
            case 'm13':
                ra = 250.4
                dec = 36.46
            case 'ngp':
                ra = 192.8595
                dec = 27.1283
            case 'sgp':
                ra = 12.8595
                dec = -27.1283
            case _:
                ra = 0.0
                dec = 0.0

    pix = healpix.get_healpix_from_skycoord(level, SkyCoord(ra=ra*u.deg, dec=dec*u.deg))
    outer_pix = healpix.get_healpix_from_skycoord(config.outer_level, SkyCoord(ra=ra*u.deg, dec=dec*u.deg))
else:
    if level < config.outer_level:
        raise ValueError(f"Level {level} must be greater than or equal to {config.outer_level}")
    elif level == config.outer_level:
        outer_pix = pix
    else:
        outer_pix = healpix.get_parent_pixel(level, pix, config.outer_level)

print(f"Outer Pixel: {outer_pix}")

asterisms = aomap.load_asterisms(config, outer_pix, ao_system_name)
stars = aomap.get_outer_stars(config, outer_pix, required_band=ao_system['band'], use_cache=True)
print(f"Total Asterisms in Outer Pixel: {len(asterisms):,.0f}")
print(f"Total Stars in Outer Pixel: {len(stars):,.0f}")

print(f"Zoom Level {level} Pixel: {pix}")

plot_level = config.max_data_level if plot_type == 'dust' else config.inner_level
min_mag = int(np.floor(ao_system['nom_mag']))
max_mag = int(np.ceil(ao_system['max_mag'])) if ao_system['nom_mag'] < ao_system['max_mag'] else 19

aomap.plot_map(aomap.get_map_data(config, plot_level, ('dust-extinction' if plot_type == 'dust' else 'star-density'), level=level, pixs=pix),
    stars=[stars, {'band': ao_system['band'], 'min_mag': min_mag, 'max_mag': max_mag, 'max_size': 50, 'fc': 'white', 'ec': 'black'}],
    asterisms=[asterisms, {'fov': ao_system['fov'], 'fov_1ngs': ao_system['fov_1ngs']}],
    projection='astro', norm='linear', width=12, boundaries_level=level+1, boundaries_pixs=0, cbar_format=('%.3f' if plot_type == 'dust' else '%d')
)